# Extending Leveraged ETFs Back In Time

### Because sometimes ETFs just don't go back far enough

#### TL;DR: [Show Me The Money](#Example) or [download the data](#Download).


![3X S&P Back to the 80s](images/upro.png)


This takes a [leveraged ETF](https://www.investopedia.com/terms/l/leveraged-etf.asp) and extends it back into the past using a proxy fund.  

The basic idea is to multiply the daily returns of the proxy by the
leverage factor, adjusting for fees and other expenses.  Since some of those expenses are hard to obtain, 
it can also find the parameters that minimize the difference between the leveraged proxy and the actual leveraged ETF.
It plots a telltale chart with difference metrics and writes the simulated prices out to CSV.

If you are new to Jupyter Notebook, you can find [tutorials](https://learn.onemonth.com/jupyter-notebook-a-beginners-tutorial/) online.  If you are not already, you can edit and run this notebook interactively on [Binder](https://mybinder.org/v2/gl/doctorj%2Fquantitative-investing/master?filepath=Leveraged%20ETFs.ipynb).

In [1]:
import sys
import warnings
import unittest     
from itertools import chain, combinations

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance
from scipy.optimize import minimize
from IPython.display import Image, display, HTML

from util import yget, read_fred, annret, annvol, cumret, TRADING_DAYS


assert sys.version_info >= (3, 6, 0), "Ordered dicts are where it's at"
display(HTML("<style>.container { width:90% !important; }</style>"))
# Set to plotly for interactive figures, matplotlib for static images
pd.options.plotting.backend = "plotly"  
if pd.options.plotting.backend == "plotly":
    import plotly.io
    plotly.io.templates.default = "plotly_white"
RASTER = True   # Rasterize complex images to save time/space    
    
%matplotlib inline

In [2]:
plt.rcParams["figure.figsize"] = (16, 9)    # Matplotlib likes this in a separate cell

### Get Borrowing Rates
Leveraged funds have to borrow money, and the borrowing cost is not included in the expense ratio, so we have to account for it.  We use the [Effective Federal Funds Rate](https://en.wikipedia.org/wiki/Federal_funds_rate) because it has history back to 1954 and gives slightly better fits than the LIBOR.

In [3]:
# Source is in percent.
fedfunds = read_fred('DFF').rename('FEDFUNDS') / 100
#tbill = read_fred('DTB3').rename('TBILL') / 100
#libor1d  = read_fred('USDONTD156N').rename('LIBOR-1d') / 100
#libor1w  = read_fred('USD1WKD156N').rename('LIBOR-1w') / 100
#libor1m  = read_fred('USD1MTD156N').rename('LIBOR-1m') / 100     
#libor12m = read_fred('USD12MD156N').rename('LIBOR-12m') / 100

BORROW = fedfunds  

### Leverage

This function leverages a proxy price series using configurable leverage factor, expense ratio, and borrowing rate.

As a first pass, to leverage a daily return $ret$ by a leverage factor $factor$, we just scale the return and subtract the (daily) expense ratio $exp$:

$$ lev = factor * ret - exp $$

However, the fund's borrowing and trading costs are *not* included in the expense ratio, so we subtract borrowing costs from the leveraged return.  A 3X fund meeds to borrow an additional 2X the principal, broadly speaking, or in general `factor - 1` times the borrowing costs.  Borrowing costs vary over time, so we use a short-term interest rate benchmark such as the Federal Funds Rate or LIBOR.

$$ lev = factor * ret - exp - (factor - 1) * borrow$$

Under the hood, leveraged ETFs hold some stock and some [swaps](https://learn.robinhood.com/articles/s3FYEQ0gYx0cNAoiG25du/what-is-a-swap/); the main unknowns in simulating an ETF are the fraction of assets in swaps and the swap rate ([some exploration here](https://www.bogleheads.org/forum/viewtopic.php?p=4884654#p4884654) and [here](https://www.bogleheads.org/forum/viewtopic.php?p=5729993#p5729993)).

To account for these unknowns, we add an adjustment $C$, to be determined for each ETF.

$$ lev = factor * ret - exp - (factor - 1) * borrow + C$$

$C$ is a constant found by curve-fitting, usually quite small.

Finally, the expense ratio and borrowing costs are annual figures, so to get daily values we divide by the periods per year.

In [4]:
def leverage(prices, factor=2, expense=0.01, borrow_rate=BORROW, ann_periods=TRADING_DAYS, 
             factor_scale=1.0, factor_incr=0, borrow_scale=1.0, expense_incr=0):
    """:Return: a Series giving the daily leveraged value of `prices` at a given leverage `factor`.
    
    This is basically the per-period change in prices minus the expense ratio and borrow rate.
    The expense ratio and borrow rate are divided (evenly, arithmetically) by `ann_periods`.
    
    :param float factor: The leverage factor by which daily returns are multiplied.
    :param float expense: Net expense ratio per `ann_periods` as a fraction. Deducted proportionally from each period.  
      Example: 0.0095 for a 0.95% annual expense ratio.
    :param Series borrow_rate: The (annualized) interest rate used to finance short-term borrowing for leverage.  
      Deducted from daily returns.  Typically the daily Federal Funds Rate or LIBOR. 
    :param int ann_periods: The number of periods over which rates are given.  E.g., 252 for daily periods in a typical trading year.
    
    The `_scale` and `_incr` parameters are adjustments to the corresponding parameters found through curve-fitting.
    """
    
    # Align borrow rates with prices
    prices = pd.Series(prices, dtype=float)
    if prices.isna().any():
        raise ValueError('NaN in prices')
    if isinstance(borrow_rate, pd.Series):
        if borrow_rate.isna().any():
            raise ValueError('NaN in borrow_rate')
        if prices.index[0] < borrow_rate.index[0]:
            raise ValueError(f'Prices start {prices.index[0]}, before borrow_rate {borrow_rate.index[0]}')
        borrow_rate = borrow_rate.reindex(index=prices.index, method='ffill') 
    name = f'{prices.name or ""}:{round(factor, 3)}X'
    
    # Curve-fitting adjustments 
    borrow_rate *= borrow_scale
    expense += expense_incr  # This functions as an additive constant for the whole equation, since exp isn't scaled ("C" above)
    
    change = prices.pct_change() * factor * factor_scale + 1      # Period-to-period changes as ratios
    # Changes less expenses and borrowing costs, evenly distributed among periods
    net_change = change - (expense + borrow_rate * (factor + factor_incr - 1)) / ann_periods
    net_change.iat[0] = prices.iat[0]   # Start leveraged series at same value so it's easily comparable
    leveraged = net_change.cumprod()
    return leveraged.rename(name)


# All possible leverage() parameters for curve fitting and their ranges
ALL_LEV_PARAMS = {'factor_scale': (0, 3), 'factor_incr': (-2, 2), 'borrow_scale': (-5, 5), 'expense_incr': (-2, 2)}
# Actually used leverage() parameters, determined in the 'Model Selection' section
LEV_PARAMS = {p: ALL_LEV_PARAMS[p] for p in ('expense_incr',)}

Quick test to make sure we're doing something right

In [5]:
def deleverage(prices, factor, expense, borrow, ann_periods=TRADING_DAYS):
    # Assumes no fudge factors
    rets = prices.pct_change() + (expense + borrow * (factor -1)) / ann_periods
    rets = rets / factor + 1
    rets.iat[0] = 1.0
    return rets.cumprod()

    
class LeverageTest(unittest.TestCase):
    def test_leverage(self):
        vecs = (
            [1.0] * 5,
            [1.01] * 5,
            np.arange(1.0, 1.1, 0.01),
            np.arange(1.0, 0.9, 0.01),
        )
        for rets in vecs:
            for factor in (1, 2, 3, 1.25):
                for expense in (0.0, 0.01, 0.001, -0.01):
                    for borrow in (0.0, 0.01, 0.02):
                        rets = pd.Series(rets)
                        prices = pd.Series(1.0).append(rets.cumprod()).reset_index(drop=True)
                        lev = leverage(prices, factor, expense, borrow)
                        delev = deleverage(lev, factor, expense, borrow)
                        pd.testing.assert_series_equal(prices, delev, check_names=False)
        
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(LeverageTest));

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/3888551932.py:22: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  prices = pd.Series(1.0).append(rets.cumprod()).reset_index(drop=True)
.
----------------------------------------------------------------------
Ran 1 test in 0.564s

OK


#### Routines for aligning and plotting series

In [6]:
def norm(prices):
    """:Return: prices normalized to start at 1.0."""
    return prices / prices.iloc[0]


def cat(*dfs, dropna=True):
    """:Return: the column-wise concatenation of a sequence of Series or DataFrames.
    
    :param bool dropna: If True, remove rows with any NaN from the result.
    """
    result = pd.concat(dfs, axis=1)
    if dropna:
        result = result.dropna()
    return result


def align(*prices, dropna=True, norm=False):
    """:Return: The `prices` Series with only the dates in common to all of them, as a sequence.
    
    :param bool norm: If True, normalize each series of prices to start at 1.0.
    """
    aligned = cat(*prices, dropna=dropna)
    if norm:
        aligned = globals()['norm'](aligned)   # Calls norm() function, since we shadowed the name. Bit naughty.
    return tuple(col for _, col in aligned.iteritems())


def telltale(reference, *dfs, **layout_kws):
    """Plot the growth of several dataframes or series `dfs` relative to a `reference` series.
    
    https://www.bogleheads.org/wiki/Telltale_chart
    """
    tell = norm(cat(reference, *dfs))
    if tell.columns.nunique() < len(tell.columns):
        raise ValueError('Column names must be unique.')
    tell = tell.apply(lambda c: c / tell.iloc[:, 0])   # straight division doesn't work for some reason
    fig = tell.plot(title=f'Telltale Chart: {", ".join(tell.columns)}')
    if pd.options.plotting.backend == "plotly":
        fig.update_layout(margin=dict(t=50), **layout_kws).show()
               
      
def plotret(*prices, title=None):
    """Nice Plotly cumulative returns plot."""
    return norm(cat(*prices)).sub(1).plot(title=title).update_layout(
        yaxis=dict(tickformat=".0%"), 
        margin=dict(t=50), 
        legend_title_text='',
        yaxis_title='Cumulative Return',
        width=950,
        height=450,
    )
    
    
def color_leverage(factor, alpha=1.0, max_factor=3):
    """:Return: a plotly color string for a given leverage `factor`, more green for more long, more red for more short."""
    intensity = int(abs(factor) / max_factor * 255)
    if factor >= 0:
        return f'rgba(0,{intensity},0,{alpha})'
    else:
        return f'rgba({intensity},0,0,{alpha})'
    
    
def rasterize(figure, raster=False, width=1100, height=600, filename=None):
    """Maybe render a plotly `figure` as a static image, to save space and time."""
    if raster:
        if filename:
            figure.write_image(filename, width=width, height=height, scale=1, engine="kaleido")
            return Image(url=f'{filename}?cache_bust={np.random.randint(100000)}')
        else:
            return Image(figure.to_image(format="png", width=width, height=height, scale=1, engine="kaleido"))
    else:
        return figure    

    
def splice(old, new):
    """Splice together `new` prices with `old` prices before them, adjusted so new prices don't change."""
    if old.index[-1] < new.index[0]:
        raise ValueError(f'Last old index {old.index[-1]} and first new index {new.index[0]} must overlap')
    if old.index[0] > new.index[0]:
        warnings.warn(f'Old has no data older than new; old starts {old.index[0]}, new starts {new.index[0]}')
        return new
    first = old.index.get_loc(new.index[0], method='ffill')  # Find previous value if no exact match
    ratio = old.iloc[first] / new.iloc[0]
    return pd.concat((old.iloc[:first] / ratio, new), verify_integrity=True).rename(new.name)

#### Error metrics

In [7]:
def rmse(a, b):
    return np.sqrt(np.mean((a - b) ** 2))


def mae(a, b):
    return np.mean(np.abs(a - b))


def rel_rmse(a, b):
    """Relative RMSE between 1.0 and b / a, both aligned and normalized to start at 1.0."""
    a, b = align(a, b, norm=True)
    return rmse(1.0, b / a)
    

def rel_mae(a, b):
    """Relative MAE between 1.0 and b / a, both aligned and normalized to start at 1.0."""
    a, b = align(a, b, norm=True)
    return mae(1.0, b / a)
    
    
def return_rmse(a, b):
    """RMSE between simple periodic returns of price series `a` and `b`."""
    a, b = align(a.pct_change(), b.pct_change())
    return rmse(a, b)


def return_mae(a, b):
    """RMSE between simple periodic returns of price series `a` and `b`."""
    a, b = align(a.pct_change(), b.pct_change())
    return mae(a, b)


# This is, as you can imagine, sensitive to the most recent price.
def cumret_diff(a, b):
    """Absolute difference between the cumulative return of `a` and `b`."""
    a, b = align(a, b)
    return abs(cumret(a) - cumret(b))


def errstats(reference, leveraged, ann_periods=TRADING_DAYS):
    """:Return: dict of error metrics between expected `reference` price series and actual `leveraged` series
    at the dates (indices) they have in common."""
    reference, leveraged = align(reference, leveraged)
    return {
        'RMSE': rel_rmse(reference, leveraged), 
        'MAE': rel_mae(reference, leveraged),
        'RETRMSE': return_rmse(reference, leveraged),
        'RETMAE': return_mae(reference, leveraged),
        'CAGR': annret(leveraged, ann_periods) - annret(reference, ann_periods), 
        'VOL': annvol(leveraged, ann_periods) - annvol(reference, ann_periods), 
        'P99': norm(leveraged).div(norm(reference)).sub(1).abs().quantile(.99),
    }


def roundvals(d, digits=4):
    """Round the values of dict `d` to `digits` digits."""
    return {k: round(v, digits) for k, v in d.items()}

#### Parameter Optimization

These functions find the leverage parameters that minimize the error between a reference Series and a leveraged proxy,
and plot the results along with error metrics.

There is the question of which error metric to optimize.  The relative RMSE, basically how well the telltale chart aligns,
seems to do the best job of minimizing all metrics (cumulative and simple, squared and absolute) across funds.  The relative RMSE takes the simulated prices divided by the actual prices, and compute the RMSE between that and 1.0, which would be the ratio if they matched perfectly.

We can get away with local minimization here (as opposed to global) because the leverage function for a single day w.r.t. the leverage fit parameters is convex nonnegative increasing, the product of such functions (i.e. the cumulative return) is convex, and norms like RMSE are also convex.

In [8]:
def find_params(reference, proxy, factor=2, expense=0.01, borrow_rate=BORROW, ann_periods=TRADING_DAYS, 
                params=LEV_PARAMS, errfunc=rel_rmse):
    """Find `params` that minimize the error between a `reference` series and its leveraged `proxy`.
    
    :param dict params: Maps parameters to `leverage()` to the range to search for optimal values.
    :param func errfunc: Error function that will be minimized; takes two price series and returns a distance metric between them.
    """
    if not params:
        return {}   # Well that was easy
    
    reference, proxy = align(reference, proxy)
    def obj(x):
        param_dict = dict(zip(params.keys(), x))   # param name: value
        return errfunc(reference, leverage(proxy, factor, expense, borrow_rate=borrow_rate, ann_periods=ann_periods, **param_dict))
    
    # Find params x that minimize obj(x)
    x0 = tuple(map(np.mean, params.values()))  # Initial guess = midpoint of bounds
    res = minimize(obj, x0, bounds=list(params.values()))
    best = dict(zip(params.keys(), res.x))   # param name: optimal value
    return best

In [9]:
def plotbest(reference, proxy, factor=2, expense=0.01, borrow_rate=BORROW, ann_periods=TRADING_DAYS, plot=True, errfunc=rel_rmse, params=LEV_PARAMS):
    """Find leverage parameters that minimize error between `reference` and leveraged `proxy`, plot a telltale
    chart, and return the new leveraged series."""
    best = find_params(reference, proxy, factor=factor, expense=expense, borrow_rate=borrow_rate, ann_periods=ann_periods, errfunc=errfunc, params=params)
    print(reference.name + ':' + proxy.name, '\tparams:', ', '.join(f'{k}={v}' for k, v in roundvals(best, 4).items()))

    # Get leveraged series with best params
    leveraged = leverage(proxy, factor=factor, expense=expense, borrow_rate=borrow_rate, ann_periods=ann_periods, **best)
    ref, lev = align(reference, leveraged, norm=True)  # Might be superfluous
    error = errstats(ref, lev, ann_periods=ann_periods)
    print(', '.join(f'{k}: {v}' for k, v in roundvals(error).items()))
    sim = leveraged[:reference.index[0]]
    simret = cumret(sim) if not sim.empty else 0
    print(f'CUMRET: sim {simret:.4f} + actual {cumret(reference):.4f} = {(simret + 1) * (cumret(reference) + 1) - 1:.4f}')
    if plot:
        telltale(ref, lev)
    return leveraged

### Example

You can leverage your own ETF by changing the tickers below.  Change `UPRO` to the leveraged ETF you want to extend, and `^SP500TR` to the index or fund it leverages.  Change the factor and expense ratio to match the leveraged fund.  Check that the telltale chart looks reasonably flat and close to 1.0.  The RMSE should be less than say .03 or so.
`leveraged` will be the simulated leveraged price series.

In [10]:
letf, proxy = yget('UPRO'), yget('^SP500TR')

In [11]:
leveraged = plotbest(letf, proxy, factor=3, expense=0.0095, plot=True);

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:25: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  return tuple(col for _, col in aligned.iteritems())


UPRO:^SP500TR 	params: expense_incr=0.0091
RMSE: 0.0088, MAE: 0.008, RETRMSE: 0.0021, RETMAE: 0.0012, CAGR: 0.0022, VOL: 0.0047, P99: 0.0173
CUMRET: sim 1.1126 + actual 25.5198 = 55.0266


In [12]:
fig = plotret(proxy.rename('S&P500'), leveraged, title='3X S&P (UPRO) Back to the 80s')
rasterize(fig, True, filename='images/upro.png')

## Leverage All The Things
Below we extend many popular LETFs in bulk.  You can add more to the list, run the notebook, and they will be included in the output.

In [13]:
# Fund: (benchmark, leverage factor, expense ratio, issuer, start year for good data (or None to use all))
FUNDS = {
    # Mutual Funds
    'RYNVX': ('^SP500TR', 1.5, .0138, 'Rydex', '2000'),
    'ULPIX': ('^SP500TR', 2, .016, 'ProFunds', '2003'),
    #'RYTPX': ('^SP500TR', -2, .0184, 'Rydex', None),  # Bad data
    #'UOPIX': ('QQQ', 2, .0159, 'ProFunds', None),  # Bad data
    #'RYVNX': ('QQQ', -2, .0187, 'Rydex', None),    # Bad data
    'UAPIX': ('IWM', 2, .0178, 'ProFunds', '2003'),  # Russel 2000
    #'RYIRX': ('IWM', -2, .0191, 'Rydex', None),    # Russel 2000; Bad data
    'UMPIX': ('MDY', 2, .0166, 'ProFunds', '2003'), # S&P MidCap 400
    #'UDPIX': ('DIA', 2, .0172, 'ProFunds', None), # Dow; bad data
    'UTPIX': ('IDU', 1.5, .0173, 'ProFunds', '2004'), # Utilities
    'REPIX': ('IYR', 1.5, .0178, 'ProFunds', '2010'), # Real Estate
    #'SRPIX': ('IYR', -1, .0178, 'ProFunds', None), # Real Estate; Bad Data
    #'RYEUX': ('FEZ', 1.25, .0182, 'Rydex', None),  # EuroSTOXX 50; no good benchmark (data)
    'DXKLX': ('IEF', 2, .0143, 'Direxion', '2013'), # ITT
    'DXKSX': ('IEF', -2, .014, 'Direxion', '2013'), # ITT
    'UNPIX': ('EFA', 2, .0178, 'ProFunds', None),  # MSCI EAFE (large - mid foreign)
    'UUPIX': ('ADRE', 2, .0178, 'ProFunds', '2009'), # Emerging Markets
    
    
    # ETFs
    'SSO': ('^SP500TR', 2, .0091, 'ProShares', '2009'), # S&P 500
    'UPRO': ('^SP500TR', 3, .0093, 'ProShares', None),
    'SPXL': ('^SP500TR', 3, .0101, 'Direxion', '2013'),
    'SH': ('^SP500TR', -1, .009, 'ProShares', '2009'),
    'SDS': ('^SP500TR', -2, .0091, 'ProShares', '2009'),
    'SPXS': ('^SP500TR', -3, .0107, 'Direxion', '2013'),
    
    'QLD': ('QQQ', 2, .0095, 'ProShares', '2009'),     # NASDAQ 100
    'TQQQ': ('QQQ', 3, .0095, 'ProShares', None),
    'PSQ': ('QQQ', -1, .0095, 'ProShares', '2009'),
    'QID': ('QQQ', -2, .0095, 'ProShares', '2009'),
    'SQQQ': ('QQQ', -3, .0095, 'ProShares', None),     # Maybe 2013?
    
    'MVV': ('MDY', 2, .0095, 'ProShares', '2010'),     # MidCap 400
    'MYY': ('MDY', -1, .0095, 'ProShares', '2010'),
    'MZZ': ('MDY', -2, .0095, 'ProShares', '2010'),
    
    'UWM': ('IWM', 2, .0095, 'ProShares', '2010'),     # Russel 2000
    'TNA': ('IWM', 3, .0112, 'Direxion', '2013'),
    'RWM': ('IWM', -1, .0095, 'ProShares', '2010'),
    'TWM': ('IWM', -2, .0095, 'ProShares', '2009'),
    'TZA': ('IWM', -3, .0107, 'Direxion', '2013'),
    
    'URE': ('IYR', 2, .0095, 'ProShares', '2010'),     # Real Estate
    'REK': ('IYR', -1, .0095, 'ProShares', '2011'),
    #'SRS': ('IYR', -2, .0095, 'ProShares', '2010'),     # Poor fit
    
    'UPW': ('IDU', 2, .0095, 'ProShares', '2009'),     # Utilities
    'SDP': ('IDU', -2, .0095, 'ProShares', '2009'),
    
    'EFO': ('EFA', 2, .0095, 'ProShares', None),       # MSCI EAFE
    'EFZ': ('EFA', -1, .0095, 'ProShares', '2010'),    
    'EFU': ('EFA', -2, .0095, 'ProShares', '2010'),
    
    'EET': ('EEM', 2, .0095, 'ProShares', None),       # Emerging Markets
    'EUM': ('EEM', -1, .0095, 'ProShares', '2009'),
    'EEV': ('EEM', -2, .0095, 'ProShares', '2011'),
    
    'UST': ('IEF', 2, .0095, 'ProShares', '2012'),     # 7-10 Yr Treasury
    'TYD': ('IEF', 3, .0109, 'Direxion', '2010'),
    'TBX': ('IEF', -1, .0095, 'ProShares', None),
    'PST': ('IEF', -2, .0095, 'ProShares', '2011'),    
    'TYO': ('IEF', -3, .0108, 'Direxion', '2011'),
    
    'UBT': ('TLT', 2, .0095, 'ProShares', None),       # 20+ Yr Treasury
    'TMF': ('TLT', 3, .0105, 'Direxion', '2011'),
    'TBF': ('TLT', -1, .0094, 'ProShares', '2011'),
    'TBT': ('TLT', -2, .0092, 'ProShares', '2011'),    
    'TMV': ('TLT', -3, .0104, 'Direxion', '2011'),

    'UGL': ('GLD', 2, .0095, 'ProShares', None),       # Gold
    'GLL': ('GLD', -2, .0132, 'ProShares', None),
    
    'UVXY': ('^VIX', 1.5, .0095, 'ProShares', None),    # Vix
}

### Get Prices

In [14]:
extras = ('VUSTX',)   # We'll use these later

In [15]:
tickers = frozenset(chain.from_iterable((fund, proxy) for fund, (proxy, *_) in FUNDS.items())) | frozenset(extras)
prices = yget(tickers)
prices

,TWM,UNPIX,URE,REPIX,^SP500TR,PSQ,SQQQ,UST,^VIX,UVXY,...,UAPIX,MZZ,UBT,DXKSX,SDS,EFZ,UMPIX,TMV,UPW,EFU
Date,,,,,,,,,,,,,,,,,,,,,
1986-05-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-05-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-05-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-12,18.820000,10.30,50.360001,33.580002,7588.899902,15.14,63.349998,46.759998,33.570000,13.25,...,47.230000,20.530001,24.809999,29.980000,54.480000,23.750000,41.529999,141.259995,53.939999,17.129999
2022-10-13,17.879999,10.74,51.900002,34.419998,7786.959961,14.79,59.110001,46.349998,31.940001,12.69,...,49.520000,19.660000,24.320000,30.200001,51.630001,23.290001,43.349998,145.059998,56.610001,16.400000
2022-10-14,18.830000,10.41,49.220001,33.119999,7602.990234,15.25,64.379997,45.990002,32.020000,13.14,...,46.889999,20.660000,23.969999,30.510000,53.930000,23.650000,41.220001,148.889999,54.869999,16.910000


In [16]:
rasterize(pd.concat((norm(p.dropna()) for _, p in prices.iteritems()), axis=1).plot(title='All funds cumulative return'), RASTER, filename='images/cumulative.png')

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/1988333407.py:1: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



## Leverage

Now for each LETF, we find the best parameters and splice the old synthetically leveraged data with the new actual data (adjusting to most recent prices match current quotes).

Error metrics:

* **RMSE**: Root mean squared error of telltale chart: RMSE(sim / actual, 1.0)
* **MAE**: Mean absolute error of telltale chart: MAE(sim / actual, 1.0)
* **RETRMSE**: RMSE of simple daily returns
* **RETMAE**: MAE of simple daily returns
* **CAGR**: simulated CAGR - actual CAGR
* **VOL**: simulated (daily) volatility - actual volatility
* **P99**: 99th percentile of (absolute value of) telltale chart deviation


In [17]:
# Collect results from fitting and leveraging each ETF
sim, err, tells = {}, {}, {}
for name, (proxy, factor, exp, _, start) in FUNDS.items():
    # Cut out initial bad data from leveraged fund, align with proxy
    lfund, proxy = prices.loc[start:, name].dropna(), prices[proxy].dropna()
    
    leveraged = plotbest(lfund, proxy, factor, exp, plot=False)
    #params = find_params(lfund.iloc[len(lfund) // 2:], proxy, factor, exp)
    #leveraged = leverage(proxy, factor, exp, **params)
    
    sim[name] = splice(leveraged, lfund)
    lfund, lev = align(prices[name].dropna(), leveraged, norm=True)   # Plot whole series including bad initial data
    tells[name] = lev.div(lfund).rename(name)
    err[name] = errstats(lfund, lev)
    print()
    
tells = pd.concat(tells, axis=1, verify_integrity=True)
err = pd.DataFrame.from_dict(err, orient='index')

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:25: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



RYNVX:^SP500TR 	params: expense_incr=0.0029
RMSE: 0.0109, MAE: 0.0087, RETRMSE: 0.0014, RETMAE: 0.0004, CAGR: 0.0009, VOL: -0.0009, P99: 0.0312
CUMRET: sim 10.5713 + actual 2.0646 = 34.4609

ULPIX:^SP500TR 	params: expense_incr=0.0087
RMSE: 0.0106, MAE: 0.0071, RETRMSE: 0.0051, RETMAE: 0.0007, CAGR: 0.0004, VOL: 0.0025, P99: 0.0322
CUMRET: sim 4.3051 + actual 7.1083 = 42.0151



/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:82: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:25: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:82: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:25: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



UAPIX:IWM 	params: expense_incr=0.0047
RMSE: 0.013, MAE: 0.0107, RETRMSE: 0.006, RETMAE: 0.0025, CAGR: 0.0011, VOL: -0.0059, P99: 0.027
CUMRET: sim -0.4423 + actual 4.0164 = 1.7979

UMPIX:MDY 	params: expense_incr=0.0009
RMSE: 0.0213, MAE: 0.0196, RETRMSE: 0.0065, RETMAE: 0.0023, CAGR: 0.0012, VOL: 0.0052, P99: 0.0301
CUMRET: sim 1.5613 + actual 7.7252 = 21.3480



/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:82: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:25: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:82: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:25: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



...

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:82: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:25: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:82: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:25: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



GLL:GLD 	params: expense_incr=0.0177
RMSE: 0.044, MAE: 0.0395, RETRMSE: 0.0019, RETMAE: 0.0011, CAGR: -0.005, VOL: -0.002, P99: 0.0814
CUMRET: sim -0.7467 + actual -0.9262 = -0.9813



/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:82: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:25: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



UVXY:^VIX 	params: expense_incr=1.7237
RMSE: 0.7863, MAE: 0.7273, RETRMSE: 0.0676, RETMAE: 0.0455, CAGR: -0.0855, VOL: 0.7728, P99: 1.1029
CUMRET: sim -1.0000 + actual -1.0000 = -1.0000



/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:82: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:25: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



In [18]:
err.eval("CAGR = abs(CAGR)\nVOL = abs(VOL)").describe()

,RMSE,MAE,RETRMSE,RETMAE,CAGR,VOL,P99
count,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000
mean,0.069782,0.065426,0.005094,0.002709,0.004863,0.021062,0.097853
std,0.116037,0.108441,0.009219,0.006178,0.011782,0.106488,0.157059
min,0.002229,0.001736,0.001265,0.000652,0.000106,0.000087,0.006657
25%,0.016098,0.013052,0.002153,0.001149,0.000954,0.001780,0.029101
50%,0.029612,0.025541,0.002754,0.001389,0.002184,0.004739,0.057271
75%,0.089625,0.085746,0.005055,0.002463,0.005195,0.008897,0.118457
max,0.786341,0.727341,0.067579,0.045505,0.085471,0.772773,1.102942


### Telltale Charts

In [19]:
lev_colors = {name: color_leverage(factor, alpha=0.2) for name, (_, factor, _, _, _) in FUNDS.items()}
fig = tells.plot(color_discrete_map=lev_colors, title='Telltale, Simulated vs. Actual Leveraged ETFs<br>In-sample fit', render_mode='webgl')\
    .update_layout(yaxis_title='Simulated / Actual', legend_title_text='Green = long<br>Red = short')
rasterize(fig, RASTER, filename='images/telltales.png')

#### Patch up TMF / VUSTX
TMF's proxy TLT only goes back to 2002, so we use VUSTX before that.  It's not an exact proxy, but better than nothing.  ¯\_(ツ)_/¯

In [20]:
_, factor, exp, _, start = FUNDS['TMF']
sim_vustx = plotbest(prices.loc[start:, 'TMF'].dropna(), prices['VUSTX'].dropna(), factor, exp);

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:25: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



TMF:VUSTX 	params: expense_incr=-0.0023
RMSE: 0.0296, MAE: 0.025, RETRMSE: 0.0047, RETMAE: 0.0027, CAGR: 0.0074, VOL: -0.035, P99: 0.0795
CUMRET: sim 12.0356 + actual -0.0140 = 11.8536


Splice VUSTX + TLT + TMF

In [38]:
sim['TMF'] = splice(sim_vustx, sim['TMF'])
plotret(cat(splice(prices['VUSTX'].dropna(), prices['TLT'].dropna()), sim['TMF']), title='The Great Bond Bull Run in One Figure')

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:82: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:82: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.



Saving the reconstructed TLT too even if it's not leveraged because it might be useful

In [42]:
sim["TLT"] = splice(prices['VUSTX'].dropna(), prices['TLT'].dropna())

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:82: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.



### Save Results to CSV

In [43]:
def is_mutual_fund(ticker):
    return len(ticker) == 5 and ticker.endswith('X')

In [44]:
filename = 'extended-leveraged-etfs.csv'
pd.concat((data for name, data in sim.items() if not is_mutual_fund(name)), axis=1, verify_integrity=True).to_csv(filename, float_format='%.5f')
!du -h $filename; echo
!head -3 $filename; echo; tail -2 $filename

3.0M	extended-leveraged-etfs.csv

Date,SSO,UPRO,SPXL,SH,SDS,SPXS,QLD,TQQQ,PSQ,QID,SQQQ,MVV,MYY,MZZ,UWM,TNA,RWM,TWM,TZA,URE,REK,UPW,SDP,EFO,EFZ,EFU,EET,EUM,EEV,UST,TYD,TBX,PST,TYO,UBT,TMF,TBF,TBT,TMV,UGL,GLL,UVXY,TLT
1986-05-19,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.69170,,,,,,,10.60539
1986-05-20,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.68715,,,,,,,10.58417

2022-10-17,41.61000,30.21000,57.00000,16.79000,51.22000,26.66000,36.97000,19.30000,14.72000,26.56000,58.00000,43.99000,26.33000,19.50000,31.07000,32.07000,25.17000,17.64000,39.35000,52.98000,21.81000,57.20000,13.86000,27.04000,23.05000,16.06000,41.67000,16.83000,28.66000,46.00000,28.62000,29.10000,22.99000,14.18000,23.68000,7.53000,23.26000,34.61000,151.49001,45.97000,37.84000,12.61000,98.09000
2022-10-18,42.57000,31.11000,58.99000,16.59000,49.96000,25.73000,37.55000,19.78000,14.59000,26.17000,56.59000,45.26000,25.96000,18.94000,31.76000,33.20000,24.87000,17.25000,37.98000,54.20000,21.53000,59.13140,13.27000,27.37010,22.87000,15.87880,,1

#### Download

In [45]:
display(HTML(f'<h3><a href="{filename}" download>Download CSV</a></h3>'))

In [25]:
assert False, "The note-buck stops here"

AssertionError: The note-buck stops here

## Appendix: Model Selection
Experiements to finds the (sub)set of curve fitting parameters that minimize the out-of-sample prediction error.

This isn't necessary to use the leveraging machinery above.

In [29]:
def oos_error(funds, prices, param_ranges):
    """Find the best leverage parameters for the last half of each fund in `funds`, then use them
    to leverage the first half of each fund and compute the error."""
    tells = {}
    params = {}
    err = {}
    for name, (proxy, factor, exp, _, start) in funds.items():
        # Cut out initial bad data from leveraged fund, align with proxy
        proxy = prices[proxy].dropna()
        lfund, _ = align(prices.loc[start:, name], proxy)  # We do *not* want to modify the proxy
        # Find best params for last half of data
        assert len(lfund) > 500, "That's not enough data!"
        mid = len(lfund) // 2
        params[name] = find_params(lfund.iloc[mid:], proxy, factor, exp, params=param_ranges)
        # Use params to leverage all data
        lev = leverage(proxy, factor, exp, **params[name])
        # Compute error on first half (out of sample)
        err[name] = errstats(lfund.iloc[:mid], lev)
        # Plot the whole thing
        lfund, lev = align(prices[name], lev, norm=True)   # Plot whole series including bad initial data
        tells[name] = lev.div(lfund).rename(name)
        
    return pd.DataFrame.from_dict(err, orient='index'), pd.DataFrame.from_dict(params, orient='index'), pd.concat(tells, axis=1, verify_integrity=True)

In [30]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [31]:
%%time
# Compute out of sample error for all subsets of fitting parameters, show median + IQR for RMSE and CAGR
# This gives (roughly) equal weight to each fund in the error
result = []
for params in powerset(ALL_LEV_PARAMS):
    combo = ' + '.join(params) or 'none'
    print(combo)
    err, params, tells = oos_error(FUNDS, prices, {p: ALL_LEV_PARAMS[p] for p in params})
    summ = err.eval("CAGR = abs(CAGR)\nVOL = abs(VOL)").describe()
    summ.loc['iqr', :] = summ.loc['75%', :] - summ.loc['25%', :]
    result.append({'params': combo,
                   'RMSE': summ.loc['50%', 'RMSE'], 'RMSE_iqr': summ.loc['iqr', 'RMSE'],
                   'CAGR': summ.loc['50%', 'CAGR'], 'CAGR_iqr': summ.loc['iqr', 'CAGR'],
    })
    #display(summ)
    #display(params)
    
result = pd.DataFrame(result)

none


/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:25: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



factor_scale


/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:25: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



factor_incr


...

factor_incr + borrow_scale + expense_incr


/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:25: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



factor_scale + factor_incr + borrow_scale + expense_incr


/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:25: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



CPU times: user 4min 31s, sys: 1.91 s, total: 4min 33s
Wall time: 4min 36s


In [32]:
result.set_index('params').style.background_gradient()

,RMSE,RMSE_iqr,CAGR,CAGR_iqr
params,,,,
none,0.026576,0.036758,0.005196,0.008121
factor_scale,0.022101,0.021855,0.003440,0.004824
factor_incr,0.029460,0.042286,0.005838,0.008374
borrow_scale,0.029460,0.042286,0.005838,0.009912
expense_incr,0.018347,0.016730,0.002984,0.004856
factor_scale + factor_incr,0.023418,0.035260,0.005023,0.008681
factor_scale + borrow_scale,0.023417,0.035258,0.005023,0.008681
factor_scale + expense_incr,0.017820,0.018331,0.002849,0.004381
factor_incr + borrow_scale,0.029460,0.042286,0.005838,0.009912


In [33]:
# Show error stats for best model / fit params
err, params, tells = oos_error(FUNDS, prices, LEV_PARAMS)

/var/folders/hf/f4l7mwpx34370yzy_vcsrgvw0000gn/T/ipykernel_10469/2792222133.py:25: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



In [34]:
err.describe()

,RMSE,MAE,RETRMSE,RETMAE,CAGR,VOL,P99
count,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000
mean,1.204467,0.749959,0.003937,0.002427,0.003363,0.009056,4.895417
std,8.526646,5.267358,0.008185,0.005672,0.023718,0.075919,35.037674
min,0.003119,0.002532,0.000763,0.000274,-0.022863,-0.061230,0.007356
25%,0.008627,0.007271,0.001311,0.000811,-0.003056,-0.001039,0.014901
50%,0.018347,0.016410,0.001595,0.001086,0.000577,0.001122,0.030902
75%,0.025357,0.022795,0.003499,0.001820,0.002891,0.003874,0.043370
max,61.508399,38.002749,0.057129,0.040564,0.164485,0.540677,252.696738


In [35]:
params.describe()

,expense_incr
count,52.000000
mean,0.017829
std,0.079338
min,-0.006599
25%,0.001338
50%,0.005591
75%,0.010513
max,0.576043


In [36]:
fig = tells.plot(color_discrete_map=lev_colors, title='Telltale, Simulated vs. Actual Leveraged ETFs<br>Out-of-sample fit')\
    .update_layout(yaxis_title='Simulated / Actual', legend_title_text='Green = long<br>Red = short')
rasterize(fig, RASTER, filename='images/telltales-oos.png')